In [3]:
import pandas as pd
import geopandas as gpd

In [82]:
## read adoption rate data
adopt_rates = pd.read_csv('../DSSG2023-Heating-Loads-Data/state-borough-aggregates/adoption_scenarios_5_and_15.csv', usecols=lambda x: x not in ['Unnamed: 0'])

In [83]:
adopt_rates.rename(columns={'NAME':'census_area'}, inplace=True)

In [84]:
adopt_rates.head()

,census_area,hp_current,hp_current_prop_b,hp_5p,hp_5p_prop_b,hp_15p,hp_15p_prop_b
0,Aleutians East Borough,0,0.000000,27,0.039764,339.5,0.500000
1,Aleutians West Census Area,0,0.000000,96,0.072126,665.5,0.500000
2,Anchorage Municipality,134,0.001129,217,0.001829,4195.0,0.035359
3,Bethel Census Area,0,0.000000,25,0.004178,1247.0,0.208389
4,Bristol Bay Borough,0,0.000000,13,0.015169,428.5,0.500000


In [85]:
## read borough estimates
borough_estimates_weightedByHuFuel = pd.read_csv('../Output/borough_estimates_weightedByHuFuel.csv', usecols=lambda x: x not in ['Unnamed: 0'])

In [86]:
## correct name inconsistency
borough_estimates_weightedByHuFuel.replace('Anchorage municipality', 'Anchorage Municipality', inplace=True)
borough_estimates_weightedByHuFuel.replace('Wade Hampton Census Area', 'Kusilvak Census Area', inplace=True)

In [87]:
## get the only base scenarios 
borough_estimates_baseScen = borough_estimates_weightedByHuFuel.loc[(borough_estimates_weightedByHuFuel['Rebate_dol'] == 0) & (borough_estimates_weightedByHuFuel['Fuel_Esc_Rate'] == 0.03) & (borough_estimates_weightedByHuFuel['Temp_Projection'] == 0)]

In [88]:
borough_estimates_baseScen.head()

,census_area,Rebate_dol,Fuel_Esc_Rate,Temp_Projection,NPV_weighted2,CO2_lbs_saved_weighted2,CO2_driving_miles_saved_weighted2,Heating_Days_Above5_weighted2,Heating_Days_Below5_weighted2,Heating_Days_Covered
0,Aleutians East Borough,0,0.03,0.0,292.098980,2202.777896,2469.844670,363.461122,0.000000,1.000000
27,Aleutians West Census Area,0,0.03,0.0,-7626.587914,2273.699688,2549.365084,355.512971,0.000000,1.000000
54,Anchorage Municipality,0,0.03,0.0,-5089.637038,1765.274706,1979.298199,354.500000,9.833333,0.973010
81,Bethel Census Area,0,0.03,0.0,-2297.664847,1085.389886,1216.983532,327.521745,34.923018,0.903646
108,Bristol Bay Borough,0,0.03,0.0,-9005.019276,1381.682427,1549.198848,288.469648,19.400006,0.936986


In [89]:
## check if the boroughs match perfectly
[borough_estimates_baseScen['census_area'].isin(adopt_rates['census_area']).mean(), adopt_rates['census_area'].isin(borough_estimates_baseScen['census_area']).mean()]

[1.0, 1.0]

In [90]:
## Merge adoption projects to borough estimates
borough_estimates_wAdoptRate = borough_estimates_baseScen.merge(adopt_rates, on='census_area')

In [91]:
borough_estimates_wAdoptRate.head()

,census_area,Rebate_dol,Fuel_Esc_Rate,Temp_Projection,NPV_weighted2,CO2_lbs_saved_weighted2,CO2_driving_miles_saved_weighted2,Heating_Days_Above5_weighted2,Heating_Days_Below5_weighted2,Heating_Days_Covered,hp_current,hp_current_prop_b,hp_5p,hp_5p_prop_b,hp_15p,hp_15p_prop_b
0,Aleutians East Borough,0,0.03,0.0,292.098980,2202.777896,2469.844670,363.461122,0.000000,1.000000,0,0.000000,27,0.039764,339.5,0.500000
1,Aleutians West Census Area,0,0.03,0.0,-7626.587914,2273.699688,2549.365084,355.512971,0.000000,1.000000,0,0.000000,96,0.072126,665.5,0.500000
2,Anchorage Municipality,0,0.03,0.0,-5089.637038,1765.274706,1979.298199,354.500000,9.833333,0.973010,134,0.001129,217,0.001829,4195.0,0.035359
3,Bethel Census Area,0,0.03,0.0,-2297.664847,1085.389886,1216.983532,327.521745,34.923018,0.903646,0,0.000000,25,0.004178,1247.0,0.208389
4,Bristol Bay Borough,0,0.03,0.0,-9005.019276,1381.682427,1549.198848,288.469648,19.400006,0.936986,0,0.000000,13,0.015169,428.5,0.500000


In [118]:
## loading borough shapefiles
boroughs = gpd.read_file('../DSSG2023-Heating-Loads-Data/shapefiles/Boroughs2020/')

In [119]:
boroughs = boroughs[['NAME', 'HOUSEUNITS']]

In [120]:
boroughs.rename(columns={'NAME':'census_area'}, inplace=True)

In [121]:
borough_estimates_wAdoptRateHU = borough_estimates_wAdoptRate.merge(boroughs, on='census_area')

In [122]:
borough_estimates_wAdoptRateHU['total_NPV_current'] = borough_estimates_wAdoptRateHU['NPV_weighted2'] * borough_estimates_wAdoptRateHU['hp_current_prop_b'] * borough_estimates_wAdoptRateHU['HOUSEUNITS']
borough_estimates_wAdoptRateHU['total_NPV_5p'] = borough_estimates_wAdoptRateHU['NPV_weighted2'] * borough_estimates_wAdoptRateHU['hp_5p_prop_b'] * borough_estimates_wAdoptRateHU['HOUSEUNITS']
borough_estimates_wAdoptRateHU['total_NPV_15p'] = borough_estimates_wAdoptRateHU['NPV_weighted2'] * borough_estimates_wAdoptRateHU['hp_15p_prop_b'] * borough_estimates_wAdoptRateHU['HOUSEUNITS']

borough_estimates_wAdoptRateHU['total_CO2_lbs_current'] = borough_estimates_wAdoptRateHU['CO2_lbs_saved_weighted2'] * borough_estimates_wAdoptRateHU['hp_current_prop_b'] * borough_estimates_wAdoptRateHU['HOUSEUNITS']
borough_estimates_wAdoptRateHU['total_CO2_lbs_5p'] = borough_estimates_wAdoptRateHU['CO2_lbs_saved_weighted2'] * borough_estimates_wAdoptRateHU['hp_5p_prop_b'] * borough_estimates_wAdoptRateHU['HOUSEUNITS']
borough_estimates_wAdoptRateHU['total_CO2_lbs_15p'] = borough_estimates_wAdoptRateHU['CO2_lbs_saved_weighted2'] * borough_estimates_wAdoptRateHU['hp_15p_prop_b'] * borough_estimates_wAdoptRateHU['HOUSEUNITS']

borough_estimates_wAdoptRateHU['total_CO2_miles_current'] = borough_estimates_wAdoptRateHU['CO2_driving_miles_saved_weighted2'] * borough_estimates_wAdoptRateHU['hp_current_prop_b'] * borough_estimates_wAdoptRateHU['HOUSEUNITS']
borough_estimates_wAdoptRateHU['total_CO2_miles_5p'] = borough_estimates_wAdoptRateHU['CO2_driving_miles_saved_weighted2'] * borough_estimates_wAdoptRateHU['hp_5p_prop_b'] * borough_estimates_wAdoptRateHU['HOUSEUNITS']
borough_estimates_wAdoptRateHU['total_CO2_miles_15p'] = borough_estimates_wAdoptRateHU['CO2_driving_miles_saved_weighted2'] * borough_estimates_wAdoptRateHU['hp_15p_prop_b'] * borough_estimates_wAdoptRateHU['HOUSEUNITS']

borough_estimates_wAdoptRateHU['total_Heating_Days_Above5_current'] = borough_estimates_wAdoptRateHU['Heating_Days_Above5_weighted2'] * borough_estimates_wAdoptRateHU['hp_current_prop_b'] * borough_estimates_wAdoptRateHU['HOUSEUNITS']
borough_estimates_wAdoptRateHU['total_Heating_Days_Above5_5p'] = borough_estimates_wAdoptRateHU['Heating_Days_Above5_weighted2'] * borough_estimates_wAdoptRateHU['hp_5p_prop_b'] * borough_estimates_wAdoptRateHU['HOUSEUNITS']
borough_estimates_wAdoptRateHU['total_Heating_Days_Above5_15p'] = borough_estimates_wAdoptRateHU['Heating_Days_Above5_weighted2'] * borough_estimates_wAdoptRateHU['hp_15p_prop_b'] * borough_estimates_wAdoptRateHU['HOUSEUNITS']

borough_estimates_wAdoptRateHU['total_Heating_Days_Below5_current'] = borough_estimates_wAdoptRateHU['Heating_Days_Below5_weighted2'] * borough_estimates_wAdoptRateHU['hp_current_prop_b'] * borough_estimates_wAdoptRateHU['HOUSEUNITS']
borough_estimates_wAdoptRateHU['total_Heating_Days_Below5_5p'] = borough_estimates_wAdoptRateHU['Heating_Days_Below5_weighted2'] * borough_estimates_wAdoptRateHU['hp_5p_prop_b'] * borough_estimates_wAdoptRateHU['HOUSEUNITS']
borough_estimates_wAdoptRateHU['total_Heating_Days_Below5_15p'] = borough_estimates_wAdoptRateHU['Heating_Days_Below5_weighted2'] * borough_estimates_wAdoptRateHU['hp_15p_prop_b'] * borough_estimates_wAdoptRateHU['HOUSEUNITS']

In [123]:
borough_estimates_totals = borough_estimates_wAdoptRateHU.filter(regex=('total_'))

In [124]:
state_estimates = borough_estimates_totals.sum()

In [132]:
state_estimates['total_Heating_Days_Covered_current'] = state_estimates['total_Heating_Days_Above5_current'] / (state_estimates['total_Heating_Days_Above5_current'] + state_estimates['total_Heating_Days_Below5_current'])
state_estimates['total_Heating_Days_Covered_5p'] = state_estimates['total_Heating_Days_Above5_5p'] / (state_estimates['total_Heating_Days_Above5_5p'] + state_estimates['total_Heating_Days_Below5_5p'])
state_estimates['total_Heating_Days_Covered_15p'] = state_estimates['total_Heating_Days_Above5_15p'] / (state_estimates['total_Heating_Days_Above5_15p'] + state_estimates['total_Heating_Days_Below5_15p'])

In [133]:
state_estimates.rename('estimate', inplace=True)

total_NPV_current                     4.408303e+07
total_NPV_5p                          6.213353e+07
total_NPV_15p                         1.697973e+08
total_CO2_lbs_current                 2.000964e+07
total_CO2_lbs_5p                      3.088174e+07
total_CO2_lbs_15p                     1.440788e+08
total_CO2_miles_current               2.243563e+07
total_CO2_miles_5p                    3.462587e+07
total_CO2_miles_15p                   1.615470e+08
total_Heating_Days_Above5_current     1.324617e+06
total_Heating_Days_Above5_5p          2.223758e+06
total_Heating_Days_Above5_15p         1.534180e+07
total_Heating_Days_Below5_current     2.606896e+04
total_Heating_Days_Below5_5p          3.661741e+04
total_Heating_Days_Below5_15p         4.604929e+05
total_Heating_Days_Covered_current    9.806995e-01
total_Heating_Days_Covered_5p         9.838003e-01
total_Heating_Days_Covered_15p        9.708591e-01
Name: estimate, dtype: float64

In [134]:
state_estimates.to_csv('../Output/state_estimates.csv')